# [admin] Volumes command


The `volumes` command in `admin` scope could help you manage volumes.


## Setup PrimeHub Python SDK


In [1]:
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
else:
    print("PrimeHub Python SDK couldn't get the group information, follow the 00-getting-started.ipynb to complete it")

PrimeHub Python SDK setup successfully


## Help documentation

In [2]:
help(ph.admin.volumes)

Help on AdminVolumes in module primehub.admin_volumes object:

class AdminVolumes(primehub.Helpful, primehub.Module)
 |  AdminVolumes(primehub: primehub.PrimeHub, **kwargs)
 |  
 |  Method resolution order:
 |      AdminVolumes
 |      primehub.Helpful
 |      primehub.Module
 |      primehub.HTTPSupport
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_group(self, id: str, group_id, writable=False)
 |      Add group connection to a volume by id
 |      
 |      :type id: str
 |      :param id: the id of a volume
 |      :type group_id: str
 |      :param group_id: group id
 |      :type writable: bool, optional
 |      :param writable: `True` if the group has write permission to the volume, and `False` otherwise, defaults to False
 |      
 |      :rtype dict
 |      :return a volume with id only
 |  
 |  create(self, config) -> list
 |      Create a volume
 |      
 |      :rtype: dict
 |      :returns: the volume
 |  
 |  delete(self, id)
 |      Delete a volume by

## PrimeHub Admin Volume Management
Old name: PrimeHub admin Dataset

In [3]:
!primehub admin volumes help

Usage: 
  primehub admin volumes <command>

Manage volumes

Available Commands:
  add-group            Add group connection to a volume by id
  create               Create a volume
  delete               Delete a volume by id
  get                  Get a volume by name
  list                 List volumes
  list-groups          List groups of a volume by id
  regen-upload-secret  Regenerate the secret of the upload server
  remove-group         Remove group connection from a volume by id
  update               Update the volume

Options:
  -h, --help           Show the help

Global Options:
  --config CONFIG      Change the path of the config file (Default: ~/.primehub/config.json)
  --endpoint ENDPOINT  Override the GraphQL API endpoint
  --token TOKEN        Override the API Token
  --group GROUP        Override the current group
  --json               Output the json format (output human-friendly format by default)


## Dataset configuration

You need a configuration `create` and `update` to operate. Here is an example to create a `pv-volume`:

```json
{
  "name": "pv-volume",
  "displayName": "the volume created by SDK",
  "description": "It is a PV volume",
  "type": "pv",
  "global": true,
  "pvProvisioning": "auto",
  "volumeSize": 1
}
```

In our system, there are 5 types for volumes: `['pv', 'nfs', 'hostPath', 'git', 'env']`. Please check the fields reference to give a proper configuration to create your own volume.


### Fields for creating or updating

| field | required | type | description |
| --- | --- | --- | --- |
| name | required | string | it should be a valid resource name for kubernetes |
| displayName | optional | string | display name for this volume |
| description | optional | string | |
| global | optional | boolean | when a volume is global, it could be seen for each group |
| type | required | string | one of ['pv', 'nfs', 'hostPath', 'git', 'env'] |
| url | conditional | string | **MUST** use with `git` type |
| pvProvisioning | conditional | string | onf of ['auto', 'manual'], **MUST** use with `pv` type. This field only uses in `CREATE` action |
| nfsServer | conditional | string | **MUST** use with `nfs` type |
| nfsPath | conditional | string | **MUST** use with `nfs` type |
| hostPath | conditional | string | **MUST** use with `hostPath` type  |
| variables | optional | dict | **MAY** use with `env` type. It is key value pairs. All values have to a string value. For example: `{"key1":"value1","key2":"value2"}`. |
| groups | optional | list of connected groups (dict) | please see the `connect` examples |
| secret | optional | dict | **MAY** use with `git` type, it binds a `secret` id to the `git` volume. The secret can be found with `primehub admin secrets list` |
| volumeSize | conditional | integer | **MUST** use with `pv` type and 'auto' provisioning. The unit is `GB`.|
| enableUploadServer | optional | boolean | it only works with one of ['pv', 'nfs', 'hostPath'] writable types |

> There is a simple rule to use fields for `UPDATE`. All required fields should not be in the payload.

For example, there is a configuration for creating env volume:

```bash
primehub admin volumes create <<EOF
{
  "name": "env-volume",
  "description": "",
  "type": "env",
  "variables": {
    "ENV": "prod",
    "LUCKY_NUMBER": "7"
  }
}
EOF
```

After removing required `name` and `type` fields, it could be used with updating:

```bash
primehub admin volumes update env-volume <<EOF
{
  "description": "make changes to the description",
  "variables": {
    "ENV": "prod",
    "LUCKY_NUMBER": "8"
  }
}
EOF
```



## Examples

You could find [more examples on our github](https://github.com/InfuseAI/primehub-python-sdk/blob/main/docs/CLI/admin/volumes.md).

In [4]:
import pandas as pd

#### CRUD: primehub volumes

In [5]:
# Create an volume
config = {
  "name": "pv-volume-by-sdk",
  "displayName": "the volume created by SDK",
  "description": "It is a PV volume",
  "type": "pv",
  "global": True,
  "pvProvisioning": "auto",
  "volumeSize": 1
}

data = ph.admin.volumes.create(config)

In [6]:
# List volumes
volumes_df = pd.DataFrame(ph.admin.volumes.list())
volumes_df[volumes_df['name'] == config['name']]

,id,name,displayName,description,type,uploadServerLink
13,pv-volume-by-sdk,pv-volume-by-sdk,the volume created by SDK,It is a PV volume,pv,None


In [7]:
# Get the volume details
ph.admin.volumes.get(data['id'])

{'id': 'pv-volume-by-sdk',
 'name': 'pv-volume-by-sdk',
 'displayName': 'the volume created by SDK',
 'description': 'It is a PV volume',
 'type': 'pv',
 'pvProvisioning': 'auto',
 'volumeSize': 1,
 'enableUploadServer': False,
 'uploadServerLink': None,
 'global': True,
 'groups': []}

In [8]:
# Update the volume config setting.
update_config = {'global': False}
data = ph.admin.volumes.update(data['id'], update_config)
ph.admin.volumes.get(data['id'])

{'id': 'pv-volume-by-sdk',
 'name': 'pv-volume-by-sdk',
 'displayName': 'the volume created by SDK',
 'description': 'It is a PV volume',
 'type': 'pv',
 'pvProvisioning': 'auto',
 'volumeSize': 1,
 'enableUploadServer': False,
 'uploadServerLink': None,
 'global': False,
 'groups': []}

In [9]:
# Delete the volume
ph.admin.volumes.delete(data['id'])

{'id': 'pv-volume-by-sdk'}

#### CRUD: volume <-> Group

In [10]:
# Create an volume
config = {
  "name": "pv-volume-by-sdk-group",
  "displayName": "the volume created by SDK",
  "description": "It is a PV volume",
  "type": "pv",
  "global": False,
  "pvProvisioning": "auto",
  "volumeSize": 1
}

data = ph.admin.volumes.create(config)

In [11]:
# Show the volume group list

volumes_df = pd.DataFrame(ph.admin.volumes.list_groups(data["id"]))
volumes_df

""


In [12]:
# Add the volume into group
groups_df = pd.DataFrame(ph.admin.groups.list())
groups_id = groups_df["id"][0]
ph.admin.volumes.add_group(data["id"], groups_id)

In [13]:
# Show the volume group list

volumes_df = pd.DataFrame(ph.admin.volumes.list_groups(data["id"]))
volumes_df

,id,name,displayName,writable
0,71ac1e32-65fa-4e8e-a735-ba282e3149b1,AnotherTestGroup,Another Test Group,False


In [14]:
# Remove the volume out of group
ph.admin.volumes.remove_group(data["id"], groups_id)

In [15]:
# Show the volume group list

volumes_df = pd.DataFrame(ph.admin.volumes.list_groups(data["id"]))
volumes_df

""


In [16]:
# Delete the volume
ph.admin.volumes.delete(data['id'])

{'id': 'pv-volume-by-sdk-group'}